In [ ]:
import numpy as np
import xmltodict
import pandas as pd
import json
import gzip
import pickle
import csv

In [ ]:
df = pd.read_hdf('papers.h5','table')
unique_names = pickle.load(open('big_names.pkl','rb'))
uniqie_confs = pickle.load(open('confs.pkl','rb'))

In [ ]:
#big_names = np.unique(np.concatenate(df.name))
unique_names = pickle.load(open('big_names.pkl','rb'))
unique_confs = pickle.load(open('confs.pkl','rb'))
faculty_affil = pd.read_csv('faculty-affiliations.csv')
ranks = pd.read_csv('ranks.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')


In [ ]:
df.shape, unique_names.shape, unique_confs.shape,faculty_affil.shape, ranks.shape
#unique_names = [str(_) for _ in unique_names]
#np.savetxt('names.txt',unique_names,encoding='utf-8',fmt='%s')

conf_idx = {}
name_idx = {}
for i in range(unique_names.shape[0]):
    authorName = unique_names[i]
    realName = aliasdict.get(authorName, authorName)
    name_idx[realName] = i
    name_idx[authorName] = i
for i in range(unique_confs.shape[0]):
    confName = unique_confs[i]
    conf_idx[confName] = i

In [ ]:
if False:
    for name in faculty_affil.affiliation.unique():
        if name in list(ranks.uni):
            print(',',end='')
        else: #print(name)
            print(name)
            print('.',end='')
    for name in faculty_affil.name:
        if name in unique_names:
            print(',',end='')
        else: 
            print(name)
            print('.',end='')
ranks[ranks.index < 16]

In [ ]:
min_year = df.year.min()
max_year = df.year.max()
span_years = max_year - min_year
year_blocks = 8
offset_years = [(i-min_year)//year_blocks for i in range(min_year,max_year+1)]
year_ind = max(offset_years)+1
conf_idx = pickle.load(open('conf_idx.pkl','rb'))
name_idx = pickle.load(open('name_idx.pkl','rb'))

In [ ]:
offset_years

In [ ]:
import scipy.sparse
X = scipy.sparse.dok_matrix((len(unique_names),year_ind*unique_confs.shape[0]))
xdict = {}

auth_years = np.ones((len(unique_names),2)) * np.array([3000,1000]) 
#y = np.zeros(len(unique_names))#scipy.sparse.dok_matrix((1,len(unique_names)))
for row in df.itertuples():
    paper_year = row[10]
    #if row['year'] < 2005:
    #    continue
    #print(row)
    #if row['conf'] == 'CoRR':
    #    continue
    conf = row[2]
    n = row[4]
    authors = row[3]
    j = year_ind*conf_idx[conf] + (paper_year-min_year)//year_blocks
    for a in authors:
        i = name_idx[a]
        if conf != 'CoRR':
            xdict[(i,j)] = 1/n + xdict.get((i,j),0)
        auth_years[i,0] = min(auth_years[i,0],paper_year)
        auth_years[i,1] = max(auth_years[i,1],paper_year)
X._update(xdict)

In [ ]:
#pickle.dump(X,open('x.pkl','wb'))
#pickle.dump(conf_idx,open('conf_idx.pkl','wb'))
#pickle.dump(name_idx,open('name_idx.pkl','wb'))
#X_dup = np.copy(X)
#for i in range(5):
#    j = year_ind*conf_idx['CoRR'] + (i-min_year)//year_blocks
#    X[:,j] = 0


settings = [-6,'hinge',15,0]
settings = [-6, 'hinge', 20, 0.15]
#settings = [-8.548444243014805, 'hinge', 25, 0.8315610452033819]
#settings = [-8.053796519336224, 'log', 15, 0.04885118622350815]
#settings = [-8.305486025332591, 'hinge', 70, 0.8854278179436874]
#settings = [-8.475194214243022, 'modified_huber', 11, 0.01707933834322717,0.1]
#settings = [-8.403912963838302, 'modified_huber', 10, 0.7195030417696724,0.1] #15, .8,4
#settings = [-8.57, 'modified_huber',22, 0.78, 0.07]
#settings =  [-7.687656839233525, 'modified_huber', 106, 0.8035188318995535,0.1] # 27, .7. 1
#settings = [-8.438322018202042, 'modified_huber', 34, 0.746304732399994,0.1] # 23 .77 2
settings = [-4.228109135925832, 'modified_huber', 31, 0.919015912687292, 0.7758551488895429] # good ranking, bad acc
#settings = [-8.778235592882929, 'modified_huber', 11, 0.5720682745534834, 0.8463472302623746] # good acc, bad ranking
settings = [-4.502004811792925, 'modified_huber', 9, 0.9680926337835235, 0.6488148993147007] # maybe balance
settings = [-5.323680251147268,'modified_huber',9,0.9873050198822073,0.4352589129929476] # 1 error and 85 acc, it says
settings = [-4.228109135925832,'modified_huber',31,	0.919015912687292,0.7758551488895429] # no errors but little acc
#settings =[-6.013005893126464,	'modified_huber',	30,	0.9485296659531548,	0.683735528500138] # errors but acc
#settings = [-5.3,'modified_huber',31,0.98,0.6] # my own model


In [ ]:
y = np.zeros(len(unique_names))
for i in range(settings[2]): #40?
    uni_name = ranks.iloc[i]['uni']
    uni_faculty = faculty_affil[faculty_affil.affiliation == uni_name]
    uni_names = np.array(uni_faculty.name)
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            y[name_idx[name]] = 1
        else:
            pass
            #print(name)

In [ ]:
#X.shape,y.reshape((-1,))
from sklearn.linear_model import SGDClassifier

X = scipy.sparse.csr_matrix(X)
print(X.shape,y.shape)
clf = SGDClassifier(settings[1],average=False,verbose=1,warm_start=True,tol=1e-5,max_iter=1,alpha=10 ** settings[0],penalty='elasticnet',l1_ratio=settings[3],epsilon=settings[4])
clf.fit(X,y)
for i in range(25):
    minv = clf.coef_[clf.coef_ > 0].min()
    maxv = clf.coef_[clf.coef_ > 0].max()
    #clf.coef_ = np.maximum(minv,clf.coef_)
    clf = clf.partial_fit(X,y)
    print(minv,maxv)
#minv = clf.coef_[clf.coef_ > 0].min()
#clf.coef_ = np.maximum(minv,clf.coef_)


In [ ]:
X = scipy.sparse.csr_matrix(X)

import csv
with open('pairwise_tiers.csv','rt') as csvfile:
    filereader = csv.reader(csvfile)
    conf_pairs = [[t.strip('\ufeff') for t in _ if len(t)!=0] for _ in filereader]
if False:
    best_cost = 1000
    best_params = []
    for itr in range(1000):
        alpha = np.random.uniform(-6.5,-3)
        hinge_log = 'hinge' if np.random.rand() < 0.5 else 'modified_huber'
        uni_rank = int(round(np.exp(np.random.uniform(np.log(5),np.log(50)))))
        l1r = np.random.uniform(0.2,1)
        eps = np.random.uniform(0.1,1)
        
        y = np.zeros(len(unique_names))
        for i in range(uni_rank): #40?
            uni_name = ranks.iloc[i]['uni']
            uni_faculty = faculty_affil[faculty_affil.affiliation == uni_name]
            uni_names = np.array(uni_faculty.name)
            for name in set([aliasdict.get(n, n) for n in uni_names]):
                if name in name_idx:
                    y[name_idx[name]] = 1
                else:
                    pass
                    #print(name)
        #X.shape,y.reshape((-1,))
        
        from sklearn.linear_model import SGDClassifier
        clf = SGDClassifier(hinge_log,average=True,verbose=0,tol=1e-5,max_iter=1,alpha= 10 ** alpha,penalty='elasticnet',l1_ratio=l1r,epsilon=eps)
        clf.fit(X,y)
        for i in range(25):
            clf.coef_ = np.maximum(0,clf.coef_)
            clf = clf.partial_fit(X,y)
        clf.coef_ = np.maximum(0,clf.coef_)

        classifier_cost = 0
        conf_ord = np.argsort(np.squeeze(clf.coef_))
        conf_rank_dict = {}
        conf_elem = len(unique_confs)*year_ind
        for i in range(len(unique_confs)*year_ind):
            idx = conf_ord[-(i+1)]
            conf_name = unique_confs[idx//year_ind]
            conf_score = clf.coef_[0,idx]
            if idx%year_ind < 3:
                continue
            if conf_score == 0:
                conf_rank_dict[conf_name] = num_elem
            else:
                conf_rank_dict[conf_name] = i
        pair_len = len(conf_pairs)//2
        neg_top = 0
        for i in range(pair_len):
            better = conf_pairs[2*i]
            worse = conf_pairs[2*i+1]
            #print(better,worse)
            for b in better:
                neg_top += clf.coef_[0,conf_idx[b]*year_ind+4] < 0
                for w in worse:
                    classifier_cost += conf_rank_dict[w] < conf_rank_dict[b]
                    #if  conf_rank_dict[w] < conf_rank_dict[b]:
                    #    print(w,conf_rank_dict[w],'\t',b,conf_rank_dict[b])
        all_choices = clf.decision_function(X)
        frac_correct = (all_choices[y.astype(np.bool)] > 0).sum()
        print(classifier_cost,[alpha,hinge_log,uni_rank,l1r,eps],frac_correct/y.sum(),neg_top,clf.coef_[0,5570])
        if classifier_cost < best_cost:
            best_cost = classifier_cost
            best_params = [alpha,hinge_log,uni_rank,l1r,eps]
            #print(best_cost,best_params)
            #8 [-3.5003506670165976, 'log', 21]
            #10 [-3.0421236321136265, 'log', 47]
            #11 [-3.954162400006843, 'log', 109]
else:
    classifier_cost = 0
    conf_ord = np.argsort(np.squeeze(clf.coef_))
    conf_rank_dict = {}
    num_elem = len(unique_confs)*year_ind
    for i in range(num_elem):
        idx = conf_ord[-(i+1)]
        conf_name = unique_confs[idx//year_ind]
        conf_score = clf.coef_[0,idx]
        if idx%year_ind < 3:
            continue
        if conf_score == 0:
            conf_rank_dict[conf_name] = num_elem
        else:
            conf_rank_dict[conf_name] = i
    pair_len = len(conf_pairs)//2
    for i in range(pair_len):
        better = conf_pairs[2*i]
        worse = conf_pairs[2*i+1]
        #print(better,worse)
        for b in better:
            for w in worse:
                classifier_cost += (conf_rank_dict[w] < conf_rank_dict[b])
                if conf_rank_dict[w] < conf_rank_dict[b]:
                    print(w,conf_rank_dict[w],'\t',b,conf_rank_dict[b])

    all_choices = clf.decision_function(X)
    frac_correct = (all_choices[y.astype(np.bool)] > 0).sum()
    print(classifier_cost,settings,frac_correct/y.sum())

In [ ]:
(all_choices > 0).sum(),y.sum(),(all_choices[y.astype(np.bool)] > 0).sum()

In [ ]:
conf_choice = ['SIGGRAPH','NIPS','3DV','HRI','ECCV (8)','ECCV (1)','Comput. Graph. Forum','Shape Modeling International','Symposium on Geometry Processing',' Computer Aided Geometric Design','ICLR','NIPS','AAAI','I. J. Robotics Res.','CVPR','International Journal of Computer Vision','Robotics: Science and Systems','ICRA','WACV','ICML','AISTATS','CoRR','SIGGRAPH Asia','ECCV','ICCV','ISER','Humanoids','3DV','IROS','CoRL','Canadian Conference on AI','ACCV ','Graphics Interface','CRV','BMVC']
ri_confs = np.zeros(len(unique_confs)*year_ind)
print(clf.intercept_)
ms = clf.coef_.mean()
ss = clf.coef_.std()
for i in range(len(unique_confs)*year_ind):
    idx = conf_ord[-(i+1)]
    conf_name = unique_confs[idx//year_ind]
    conf_score = clf.coef_[0,idx]
    if conf_name in conf_choice:
        ri_confs[idx] = 1
    if conf_name in conf_choice and (idx%year_ind)==5:
    #if 'ICCV' in conf_name and (idx%year_ind)==4:
        start_year = offset_years.index(idx%year_ind) + 1970
        end_year = len(offset_years) - 1 - offset_years[::-1].index(idx%year_ind) + 1970
        print_name =conf_name + '_' + str(start_year)[-2:] +'t' + str(end_year)[-2:]
        print('{:20s}\t{:.0f}\t{:.1f}'.format(print_name[:20],100*conf_score,(conf_score-ms)/ss))
    ri_confs.shape,ri_confs.sum(),X.shape

In [ ]:
scores = clf.decision_function(X)
years_working = (1+auth_years[:,1]-auth_years[:,0])
value_scores = scores -  clf.intercept_[0]
norm_scores = (value_scores)/years_working


In [ ]:
ri_filter_mat = scipy.sparse.diags(ri_confs)
ri_scores = clf.decision_function(X.dot(ri_filter_mat))-clf.intercept_[0]
ri_norm_scores = ri_scores/years_working

In [ ]:
prev_cand = ['Pulkit Agrawal',
 'Joydeep Biswas',
 'Katherine L. Bouman',
 'David Braun',
 'Jia Deng',
 'Naomi T. Fitter',
 'David F. Fouhey',
 'Saurabh Gupta',
 'Judy Hoffman',
 'Hanbyul Joo',
 'Honglak Lee',
 'Changliu Liu',
 'Petter Nilsson',
 "Matthew O'Toole",
 'Alessandro Roncone',
 'Alanson P. Sample',
 'Manolis Savva',
 'Adriana Schulz',
 'Amy Tabb',
 'Fatma Zeynep Temel',
 'Long Wang',
 'Cathy Wu',
 'Ling-Qi Yan']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for ns, name in sorted([(value_scores[name_idx[ni]],ni) for ni in prev_cand],reverse=True):
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,norm_scores[ni],value_scores[ni],ri_scores[ni],years_working[ni]))
print('')
curious_names = ['Xiaolong Wang 0004','Kumar Shaurya Shankar',
                 'Nicholas Rhinehart',
                 'Humphrey Hu',
                 'David F. Fouhey',
                 'Lerrel Pinto',
                 'Justin Johnson',
                 'Amir Roshan Zamir',
                 'Brian Okorn']
print('{:20s}\t{:4s}\t{:4s}\t{:4s}\t{}'.format('name','rate','total','ri','years'))
for name in curious_names:
    ni = name_idx[name]
    print('{:20s}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.0f}'.format(name,norm_scores[ni],value_scores[ni],ri_scores[ni],years_working[ni]))

In [ ]:
if False:
    uni_faculty = faculty_affil[faculty_affil.affiliation == 'Carnegie Mellon University'] #Carnegie Mellon University
    uni_names = np.array(uni_faculty.name)
    uni_names = list(uni_names) + ['Jacob Walker','Lerrel Pinto','Brian Okorn','Leonid Keselman','Siddharth Ancha']
    cmu_scores = []
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            #if ri_scores[name_idx[name]] < 2.5:
            #    continue
            score = scores[name_idx[name]]
            cmu_scores.append((score,name))
        else:
            pass
            #print(name)
    for s,p in sorted(cmu_scores,reverse=True):
        print('{:20s}\t{:.1f}'.format(p,s))
    print('\nvalue based!\n')
    cmu_scores = []
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            #print(name,auth_years[name_idx[name]])
            score = (scores[name_idx[name]]-clf.intercept_)[0]/(1+auth_years[name_idx[name],1]-auth_years[name_idx[name],0])
            #if ri_scores[name_idx[name]] < 2.5:
            #    continue 
            cmu_scores.append((score, value_scores[name_idx[name]] ,name))

        else:
            pass
            #print(name)
    for s,rs,p in sorted(cmu_scores,reverse=True):
        print('{:20s}\t{:.1f}\t{:.1f}'.format(p,rs,s))

if True:
    print('\n best overall \n')
    cmu_scores = []

    best_scores = np.argsort(value_scores)[::-1]
    #print(best_scores.shape,unique_names[best_scores[0]])
    fa_list = list(faculty_affil.name)
    fa_a_list = list(faculty_affil.affiliation)
    uni_names = [unique_names[i] for i in best_scores[:20000]]
    for name in set([aliasdict.get(n, n) for n in uni_names]):
        if name in name_idx:
            uni = 'unknown'
            if name in fa_list:
                uni = fa_a_list[fa_list.index(name)]
            if name not in ['Jacob Walker','Justin Johnson','Pieter Abbeel','Martial Hebert','Jessica K. Hodgins','Abhinav Gupta','Christopher G. Atkeson','Tom M. Mitchell','Matthew T. Mason']:
                if years_working[name_idx[name]] < 3:
                    continue
                if years_working[name_idx[name]] > 8:
                    continue
                #if ri_scores[name_idx[name]] < 5:
                #    continue
                if auth_years[name_idx[name],1] < 2016:
                    continue
            #if (np.array(X[name_idx[name],:].todense()) * ri_confs).sum() == 0:
            #    continue
            #print(name,auth_years[name_idx[name]])
            score = norm_scores[name_idx[name]]
            ri_vscore = ri_norm_scores[name_idx[name]]
            vscore = value_scores[name_idx[name]]
            cmu_scores.append((score,ri_scores[name_idx[name]],vscore,uni,name,auth_years[name_idx[name]]))
        else:
            pass
            #print(name)
    print('{:22s}\t{:15s}\t{:5s}\t{:3s}\t{:4s}\t{} {}'.format('name','uni','rate','RI-t','total','start','end'))
    for s,ris,vs,u,p,yrs in sorted(cmu_scores,reverse=True):
        print('{:22s}\t{:15s}\t{:.3f}\t{:.1f}\t{:.2f}\t{} {}'.format(p[:22],u[:15],s,ris,vs,int(yrs[0]),int(yrs[1])))

if False:
    gz = gzip.GzipFile('dblp.xml.gz')
    names = set()
    places = set()
    papers = []
    def handle_article(_, article):
        try:
            if 'author' not in article:
                return True
            if type(article['author']) != list:
                authors = [article['author']]
            else:
                authors = article['author']

            if 'booktitle' in article:
                venue = article['booktitle']
            elif 'journal' in article:
                venue = article['journal']
            else:
                return True
            names = names.union(set(authors))
            places.add(venue)
        except:
            print(sys.exc_info()[0])
            failures += 1
            raise
        return True

    xmltodict.parse(gz, item_depth=2, item_callback=handle_article)

In [ ]:
ls *.pkl

In [ ]:
pickle.dump(clf.coef_,open('clf_faculty_neg_pos_light_reg.pkl','wb'))
